# Pinecone Vector Store - Metadata Filter

In [17]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Build a Pinecone Index and connect to it

In [18]:
import pinecone

api_key = os.environ['PINECONE_API_KEY']
pinecone.init(api_key=api_key, environment="eu-west1-gcp")

In [ ]:
# dimensions are for text-embedding-ada-002
pinecone.create_index("quickstart-index", dimension=1536, metric="euclidean", pod_type="p1")

In [19]:
pinecone_index = pinecone.Index("quickstart-index")

Build the PineconeVectorStore and GPTVectorStoreIndex

In [20]:
from llama_index import GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores import PineconeVectorStore

In [21]:
from llama_index.data_structs.node import Node

nodes = [
    Node('The Shawshank Redemption', extra_info={
        "author": "Stephen King",
        "theme": "Friendship",
    }),
    Node('The Godfather', extra_info={
        "director": "Francis Ford Coppola",
        "theme": "Mafia",
    }),
    Node("Inception", extra_info={
        "director": "Christopher Nolan",
    })
]

In [25]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index, namespace='test_05_12')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex(nodes, storage_context=storage_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 39 tokens
> [build_index_from_nodes] Total embedding token usage: 39 tokens
> [build_index_from_nodes] Total embedding token usage: 39 tokens


Define metadata filters

In [26]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters
filters = MetadataFilters(filters=[ExactMatchFilter(key='theme', value='Mafia')])

Retrieve from vector store with filters

In [27]:
retriever = index.as_retriever(filters=filters)
retriever.retrieve('What is inception about?')

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
> [retrieve] Total embedding token usage: 5 tokens
> [retrieve] Total embedding token usage: 5 tokens


[NodeWithScore(node=Node(text='director: Francis Ford Coppola\ntheme: Mafia\n\nThe Godfather', doc_id='9f345e36-ac30-4920-a85c-820ddc1276b7', embedding=None, doc_hash='8f71e3c3dcd974dc46db0991bf65028de1aaefe6229e9d46cb958bb9656aa4f4', extra_info={'director': 'Francis Ford Coppola', 'doc_id': 'None', 'document_id': 'None', 'id': '9f345e36-ac30-4920-a85c-820ddc1276b7', 'ref_doc_id': 'None', 'text': 'director: Francis Ford Coppola\ntheme: Mafia\n\nThe Godfather', 'theme': 'Mafia'}, node_info={}, relationships={}), score=0.768315315)]